In [ ]:
import robotic as ry
import numpy as np
import time

In [ ]:
config = ry.Config()
config.addFile("puzzles/p5-wall-easy.g")

In [ ]:
EGO_NAME = "ego"
OBJ_NAME = "obj"
SUB_GOAL_NAME = "sub-goal1"
GOAL_NAME = "goal"
CAMERA_NAME = "camera_top"
FLOOR_NAME = "floor"

In [ ]:
config.view()
f = config.getFrame('camera_top')
config.view_setCamera(f)

In [ ]:
q_start = config.getFrame(EGO_NAME).getPosition()[:2]
q_obj = config.getFrame(OBJ_NAME).getPosition()[:2]
q_goal = config.getFrame(GOAL_NAME).getPosition()[:2]

In [ ]:
def is_in_line_of_sight(position : np.array, target_frame : ry.Frame, config : ry.Config) -> bool:
    sight_ray = target_frame.getPosition()[:2] - position
    copy_config = config.copy()
    # TODO make the sphere bigger
    copy_config.addFrame("point_check") \
        .setShape(ry.ST.sphere, [.1]) \
        .setPosition(position.tolist() + [0.02]) 
    
    # lerp the ray to the target from object
    is_in_sight = True
    t = 0.0
    while t < 0.95:
        t += 0.05
        point = position + t * sight_ray
        copy_config.getFrame("point_check").setPosition(point.tolist() + [0.02])
        if copy_config.getCollisionFree() == False:
            is_in_sight = False
            break
        
    del copy_config
    return is_in_sight

In [ ]:
def get_proximity_score(position : np.array, goal_frame : ry.Frame) -> int:
    distance = np.linalg.norm(position - goal_frame.getPosition()[:2])
    if distance < 0.2:
        return 5
    elif distance < 0.4:
        return 2
    else:
        return 0

In [ ]:
def get_subgoal_score(
        position : np.array,
        agent_frame : ry.Frame,
        goal_frame : ry.Frame,
        config : ry.Config) -> int:
    return 10 * int(is_in_line_of_sight(position, goal_frame, config)) + 5 * int(is_in_line_of_sight(position, agent_frame, config)) + get_proximity_score(position, goal_frame)

In [ ]:
def select_node(configs : list) -> int:
    best_score = 0
    best_config = configs[0]
    for config in configs:
        score = get_subgoal_score(config.getFrame(OBJ_NAME), config.getFrame(EGO_NAME), config.getFrame(GOAL_NAME), config)
        if score > best_score:
            best_score = score
            best_config = config
    return best_config

In [ ]:
def solve(config : ry.Config, goal_frame : ry.Frame) -> list:
    # TODO implement solve function
    return None

In [ ]:
# return the path to the goal
def sub_solve(config : ry.Config, goal_frame : ry.Frame) -> list:
    # TODO implement sub_solve function
    return None

In [ ]:
def reachable(config : ry.Config, object_frame : ry.Frame) -> bool:
    rrt = ry.PathFinder()
    rrt.setProblem(config, [config.getJointState()], [object_frame.getPosition[:2]])
    ret = rrt.solve()
    del rrt
    return ret.feasible

In [ ]:
POINT_COUNT = 100
THRESHOLD = 10
SUBSET_SIZE = 10
def propose_subgoals(config : ry.Config, object_frame : ry.Frame) -> list:
    generated_points = np.random.uniform(low=-2, high=2, size=(POINT_COUNT, 2)) # random point sampling

    filtered_points = []
    for point in generated_points:
        config.getFrame(SUB_GOAL_NAME).setPosition([point[0], point[1], 0])
        score = get_subgoal_score(config.getFrame(OBJ_NAME), config.getFrame(EGO_NAME), config.getFrame(SUB_GOAL_NAME), config)
        if score >= THRESHOLD:
            filtered_points.append(point)

    subset_size = min(len(filtered_points), SUBSET_SIZE)
    point_subset = np.random.choice(filtered_points, size=subset_size, replace=False)
    return point_subset

In [ ]:
# pseudocode for forward subproblem search algorithm
# TODO correct implementation
def forward_subproblem_search(start_config : ry.Config, goal_frame : ry.Frame):
    L = [start_config]
    while len(L) > 0:
        x = select_node(L)
        path = solve(x, goal_frame)
        if path is not None: # if found
            return path # path_full (?)
        for object in objects:
            if not reachable(x, object):
                continue
            subgoals = propose_subgoals(x, object)
            for subgoal in subgoals:
                # goal = object to subgoal
                # path = sub_solve(x, goal)
                # if path found and not rej(L, x^f) then
                #   insert(L, x^f) # add last elememnt of path to L
                pass
    return None